# Reinforcement Learning with:
- Monte Carlo Simulation Algorithm
- SARSA Algorithm
- Q Learning Algorithm

# Environment and Mission 

*The goal is for the agent to learn how to navigate the state space to reach the end goal of retrieving the frisbee*
<br></br>

<U>**Within Action Space, the following actions are defined:**</U>

**'L':** Move left

**'D':** Move down

**'R':** Move right

**'U':** Move up

*If agent attempts to leave the grid, when at the edges, program would set the new state as the old state. Basically it will not move
<br></br>

<U>**Map**:</U>
    
    S  .  .  .
    
    .  H  .  H
    
    .  .  .  H
    
    H  .  .  E
<br></br>
<U>**Rewards**:</U>

Reach goal: +1

Reach hole: -1

Traversing frozen surface: 0 


---

# Building Environment


### Importing relevant packages

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random
import statistics as st
import pandas as pd

### Creating Grid Environment

#### Creating Grid Class

In [22]:
class Grid:

    # Takes in variables of rows, cols and start state of agent
    def __init__(self, rows, cols, start):
        self.rows = rows
        self.cols = cols
        self.i = start[0]
        self.j = start[1]
    
    # Fucntion that allows user to set rewards and actions allowed at given states
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions

    # Function that allows user to set state of agent
    def set_state(self,s):
        self.i = s[0]
        self.j = s[1]
    
    # Function that fetches current state of agent
    def current_state(self):
        return(self.i, self.j)
    
    # Function that checks if agent is in a terminal state (if current state of agent is in a terminal state: hole / goal state, then function returns True)
    def is_terminal(self, s):
        return s not in self.actions
    
    # Function that fetches the possible actions the agent can take at a given state s
    def possible_actions(self, s):
        return self.actions[s]
    
    # Moves the agent in the state space based on the action taken by the agent
    def move(self, action):
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
    
    # Gets reward of the current state of agent
    def get_rewards(self):
        reward = self.rewards.get((self.i, self.j), 0)
        return reward
    
    # Undo move of agent (Function isn't used but put in place if needed)
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        assert(self.current_state() in self.all_states())
    
            
    # To reset agent to be at starting state - (0, 0) in our specific example
    def reset(self):
        self.set_state((0,0))

#### Creating Grid Environment Function


In [23]:
def standard_grid(rewards, actions, rows, cols, start_state):
    # define a grid that describes the reward for arriving at each state
    # and possible actions at each state
    # the grid looks like this
    # S means start position
    # E means the end states

        # S  .  .  .
        # .  H  .  H
        # .  .  .  H
        # H  .  .  E

    g = Grid(rows, cols, start_state) #(rows, cols, start_state)
    g.set(rewards, actions)
    return g

#### Creating Environment

In [24]:
# Environment Characteristics
# no. of rows & cols of grid
no_of_rows = 4
no_of_cols = 4

# Full action space
action_space = ('D', 'U', 'L', 'R')

# Assigned start state
start_state = (0, 0)

# Define rewards at specific states (punishment yields negative rewards)
# rewards at given states (in dictionary form)
rewards = {(1, 1): -1, # hole
           (1, 3): -1, # hole
           (2, 3): -1, # hole
           (3, 0): -1, # hole
           (3, 3): 1} # frisbee

# Define legal (possible) actions at each state
# States that depict terminal state (hole / end goal) are commented because this will tie in with the .is_terminal() function under class Grid
actions = {
        (0, 0): ('D', 'R'), # Start_state
        (0, 1): ('D', 'R', 'L'), 
        (0, 2): ('D', 'R', 'L'),
        (0, 3): ('D', 'L'),
        (1, 0): ('D', 'R', 'U'),
        #(1, 1): ('D', 'R', 'L', 'U'), #Hole
        (1, 2): ('D', 'R', 'L', 'U'),
        #(1, 3): ('D', 'U', 'L'), #Hole
        (2, 0): ('D', 'U', 'R'),
        (2, 1): ('D', 'R', 'L', 'U'),
        (2, 2): ('D', 'R', 'L', 'U'),
        #(2, 3): ('D', 'U', 'L'), #Hole
        #(3, 0): ('U', 'R', ), #Hole
        (3, 1): ('U', 'R', 'L'),
        (3, 2): ('U', 'R', 'L'),
        #(3, 3): (), #End-State (frisbee)
}


# Create 4x4 Grid environment
env1 = standard_grid(rewards, actions, no_of_rows, no_of_cols, start_state) 
# Set rewards and actions of environment
env1.set(rewards, actions)
# Reset environment to start state defined as (0,0) in .reset() function
env1.reset()

##### --- Function testing ---

In [27]:
# Test if .is_terminal() function works
    # Terminal States: 1,1  1,3  2,3  3,0  3,3
print(env1.is_terminal((2, 0)))
print(env1.is_terminal((3, 0)))


# Test .move()
env1.reset()
state_before = env1.current_state()
action = env1.move('D')
state_after = env1.current_state()
print('Original State: {}, After taking action: {}'.format(state_before, state_after))


# Test loop to stop moving when environment reaches terminal state
while env1.is_terminal(env1.current_state()) == False:
    a = action_space[(random.randint(0, (len(action_space)-1)))]
    state_b = env1.current_state()
    env1.move(a)
    state_a = env1.current_state()
    
    print("State before: {}, State After taking action '{}': {}".format(state_b, a, state_a))

else:
    print('Reached terminal state {}'.format(env1.current_state()))

False
True
Original State: (0, 0), After taking action: (1, 0)
State before: (1, 0), State After taking action 'U': (0, 0)
State before: (0, 0), State After taking action 'D': (1, 0)
State before: (1, 0), State After taking action 'L': (1, 0)
State before: (1, 0), State After taking action 'R': (1, 1)
Reached terminal state (1, 1)


# Q table, Returns table and Policy

### Q table Function

*Q table is built as a dataframe for easier referencing: there were problems with referencing when building a multi nested dictionary*

In [28]:
def create_qtable(no_of_rows, no_of_cols, action_space):
    # Creates Q table as a nested dictionary
    Q = {}
    for i in range(no_of_rows):
        for j in range(no_of_cols):
            Q[(str(i) + str(j))] = 0
    
    action_space_dic = {}
    for item in action_space:
        action_space_dic[item] = 0
        

    for k, v in Q.items():
        Q[k] = action_space_dic
    
    # Converts Q table into a dataframe
    Q = pd.DataFrame(data = Q)
        
    return Q

### Returns table Function


*Returns table is built as a dataframe for easier referencing: there were problems with referencing when building a multi nested dictionary*

In [29]:
def create_returnstable(no_of_rows, no_of_cols, action_space):
    # Creates Returns table as a nested dictionary
    returns = {}
    for i in range(no_of_rows):
        for j in range(no_of_cols):
            returns[(str(i) + str(j))] = 0
    
    action_space_dic = {}
    for item in action_space:
        action_space_dic[item] = []
        

    for k, v in returns.items():
        returns[k] = action_space_dic
    
    # Converts Returns table into a dataframe
    returns = pd.DataFrame(data = returns)
        
    return returns

### Epsilon Greedy Policy

In [30]:
# Select an action for the agent to take. Each action has a minimum probability of (epsilon / no. of actions) of being selected
# Optimal action has a higher probability of being selected
def epsilon_soft(Qtable, env, epsilon, currentstate):

    prob = epsilon # sum of minimum prob of selecting all actions in action space

    # Set a random probability to determine which actions are being selected
    random_prob = random.random()
    best_actions = []
    valid_q_values = []

    # Finding max q value at the specific state for legal actions
    state_f = str(currentstate[0]) + str(currentstate[1]) # Formatted state
    for legal_action in env.actions[currentstate]:
        valid_q_values.append(Qtable.at[legal_action, state_f])
        max_q_value = max(valid_q_values)

    # Appending max q value of legal actions to best_actions list
    for item in Qtable[Qtable[state_f] == max_q_value].index.values:
        if item in env.actions[currentstate]:
            best_actions.append(item)
        else:
            continue
    
    # When random_prob =< sum of min prob of all actions, randomly select action
    if random_prob <= prob:
        # Select random action from legal actions
        valid_actions = env.actions[currentstate]
        action = valid_actions[(random.randint(0, (len(valid_actions)-1)))]
        return action
            
    # If random_prob > prob, then select legal action with highest q value
        # Other problems that this code solves:
            # 1. When more than 1 action has the same q value - select the action randomly
    else:
        action = best_actions[random.randint(0, len(best_actions)-1)]
        return action

_______________________________________________________________________________________________________________________________

# First-visit Monte Carlo Without Exploring

###  Algorithm Class

In [31]:
class monte_carlo_sim:
    '''
    ** Please remember to reset Q and Returns table after simulation

        Functions:
        # Fetches Qtable ->                            .fetchQtable()    
        # Fetches Returns Table ->                     .fetchReturnstable() 
        # Run simulation ->                            .simulate(no_of_episodes, epsilon, gamma)
        # Resets Qtable and Returns Table ->           .resettables()      
    '''
    def __init__(self, env):
        self.env = env
        # Create Q and Returns table on creating class
        self.Qtable_monte = create_qtable(self.env.rows, self.env.cols, action_space)
        self.Returns = create_returnstable(self.env.rows, self.env.cols, action_space)

    # Returns Qtable
    def fetchQtable(self):
        return self.Qtable_monte
    
    # Returns Returns table
    def fetchReturnstable(self):
        return self.Returns
    
    # Reset Q and Returns table by creating empty tables
    def resettables(self):
        self.Qtable_monte = createQtable()
        self.Returns = createReturnstable()

    # Run monte carlo simulation
    def simulate(self, no_of_episodes, epsilon, gamma):
        for i in range(no_of_episodes):
            episode = []
            G = 0

            # Reset environment to start state for each episode
            self.env.reset()

            # Loop so agent moves through state space until it reaches a terminal state (hole / end goal)
            # Store episode path (states, actions and rewards) in episode list
            while self.env.is_terminal(self.env.current_state()) == False:
                state = self.env.current_state()
                # print(state)
                action = epsilon_soft(self.Qtable_monte, self.env, epsilon, state)
                # Move agent based on selected action
                
                self.env.move(action)
                # Retrieve reward
                reward = self.env.get_rewards()

                # Append all results to episode list so that backpropogation of rewards can be done later
                episode.append((state, action, reward))

            # Reverse episode list so looping is easier
            episode_reversed = episode[::-1]
            # Create a temporary list to use for checking if there are repeated visits to states
            temp_lst = [item[0] for item in episode_reversed]

            # Loop through episodes in reverse (T-1 -> T-2 -> ... -> 0)
            for i in range(len(episode_reversed)):
                state = episode_reversed[i][0]
                state_f = str(state[0]) + str(state[1]) # state but formatted for referencing in dataframe
                act_taken = episode_reversed[i][1]
                r = episode_reversed[i][2]

                G = gamma*G + r

                # print(episode_reversed)

                # For first visit to state, append G to the Returns dataframe
                if state not in temp_lst[i+1:]:
                    self.Returns.at[act_taken, state_f] = self.Returns.at[act_taken, state_f] + [G]
                    # print(Returns.at[act_taken, state_f])
                else:
                    # Else, move on to next step in epsisode
                    continue
            
            # Update Q table with average returns for each state and action
            for state in self.Qtable_monte.columns.values:
                for action in self.Qtable_monte.index.values:
                    if len(self.Returns.at[action, state]) != 0:
                        self.Qtable_monte.loc[action, state] = st.mean(self.Returns.at[action, state])
                    else:    
                        continue


###  Running Simulation

#####  Defining Parameters

In [32]:
epsilon_monte = 0.1 # Epsilon greedy probability 
gamma_monte = 0.9 # Rewards discount rate gamma
no_of_episodes_monte = 1000 # Number of episodes to be executed in simulation

#####  Creating instance of class & running simulation

In [33]:
# Create instance of class with environment 1 (4x4 grid)
monte = monte_carlo_sim(env1)

# Run Monte Carlo simulation
monte.simulate(no_of_episodes_monte, epsilon_monte, gamma_monte)

#####  Qtable Results for *Monte Carlo Simulation*

In [34]:
monte.fetchQtable()

,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
D,0.417378,-1.000000,-0.428551,-1.00000,0.514326,0,-0.900000,0,-1.000000,0.747672,0.892473,0,0,0.000000,0.000000,0
U,0.000000,0.000000,0.000000,0.00000,0.008802,0,-0.084387,0,0.427347,-1.000000,0.047179,0,0,0.729000,0.697313,0
L,0.000000,0.039713,-0.072022,-0.28243,0.000000,0,-1.000000,0,0.000000,0.525239,0.682830,0,0,-1.000000,0.798600,0
R,-0.008031,-0.085769,-0.900000,0.00000,-1.000000,0,-1.000000,0,0.618794,0.742308,-1.000000,0,0,0.884802,1.000000,0


_______________________________________________________________________________________________________________________________

# SARSA with ϵ-Greedy Behavior Policy

###  Algorithm Class

In [103]:
class SARSA_sim:
    '''
    ** Please remember to reset Q and Returns table after simulation

        Functions:
        # Fetches Qtable ->                            .fetchQtable()    
        # Run simulation ->                            .simulate(no_of_episodes, epsilon, gamma)
        # Resets Qtable  ->                            .resettable()      
    '''
    def __init__(self, env):
        self.env = env
        # Create Q table upon creating class
        self.Qtable_sarsa = create_qtable(self.env.rows, self.env.cols, action_space)
        
    # Returns Qtable    
    def fetchQtable(self):
        return self.Qtable_sarsa
    
    # Resets Qtable by creating an empty table
    def resettable(self):
        self.Qtable_sarsa = createQtable()
    
    # Run SARSA simulation
            # There will be a sub step simulation within the overarching simulation - for looking ahead and updating Qtable
    def simulate(self, no_of_episodes, epsilon, gamma, alpha):        
        for i in range(no_of_episodes):    
            # reset environment to start state
            self.env.reset()  

            # Initiialise main simulation state
            main_step_state = self.env.current_state()

            # Loop for main simulation
            while self.env.is_terminal(self.env.current_state()) == False:
                # Assign state so that it can be referenced again later after sub step simulation is conducted
                main_step_state = self.env.current_state()

                # Loop for sub simulation - Looking ahead until agent reaches terminal state 
                while self.env.is_terminal(self.env.current_state()) == False:
                    sub_step_state = self.env.current_state()
                    state_formatted = str(sub_step_state[0]) + str(sub_step_state[1])
                    # Choose action in sub step simulation
                    sub_step_action = epsilon_soft(self.Qtable_sarsa, self.env, epsilon, sub_step_state)
                    
                    # Taking action - to observe next state, action and rewar
                    self.env.move(sub_step_action)

                    # Retrieve reward for taking specific action
                    reward = self.env.get_rewards()
                    # Retrieve new state
                    new_sub_step_state = self.env.current_state()
                    new_state_formatted = str(new_sub_step_state[0]) + str(new_sub_step_state[1])

                    # print("Sub State: {}\nSub Action: {}\n\n".format(sub_step_state, sub_step_action)) ## Test print

                    # Check if new state is a terminal state: If not, then retrieve action for new state
                    if new_sub_step_state in self.env.actions:
                        new_sub_step_action = epsilon_soft(self.Qtable_sarsa, self.env, epsilon, new_sub_step_state)
                        
                        # Update Q(s1, a1) in direction of Q(s2, a2)
                        self.Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha * (reward + (gamma * self.Qtable_sarsa.at[new_sub_step_action, new_state_formatted]) - self.Qtable_sarsa.at[sub_step_action, state_formatted]))

                    # If state is a terminal state, then update Q(s1, a1) with Q(s2, a2) = 0 (since terminal state)
                    else:
                        self.Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha * (reward + (gamma * 0) - self.Qtable_sarsa.at[sub_step_action, state_formatted]))
                        break # break out of sub simulation since it has reached terminal state

                # Since we now want to revert back to the main simulation, we will need to reassign agent back to the main state  
                self.env.set_state(main_step_state)
                # Choose action based on policy with new Q values updated by sub simulation
                main_step_action = epsilon_soft(self.Qtable_sarsa, self.env, epsilon, main_step_state)
                # Move agent
                self.env.move(main_step_action) 

                # print("Main state: {}\nMain Action: {}".format(main_step_state, main_step_action)) ## Test print

###  Running Simulation

#####  Defining Parameters

In [119]:
epsilon_sarsa = 0.1 # Epsilon greedy probability 
gamma_sarsa = 0.9 # Rewards discount rate gamma
alpha_sarsa = 0.3 # Learning rate of agent
no_of_episodes_sarsa = 800 # Number of episodes to be executed in simulation

#####  Creating instance of class & running simulation

In [120]:
# Create instance of class with environment 1 (4x4 grid)
sarsa = SARSA_sim(env1)

# Run Monte Carlo simulation
sarsa.simulate(no_of_episodes_sarsa, epsilon_sarsa, gamma_sarsa, alpha_sarsa)

#####  Qtable Results for *Q Learning with ϵ-Greedy Behavior Policy*

In [121]:
sarsa.fetchQtable()

,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
D,-0.239241,-1.000000,0.238777,-0.999202,-0.173976,0,0.809942,0,-1.000000,0.350215,0.900000,0,0,0.000000,0.000000,0
U,0.000000,0.000000,0.000000,0.000000,0.233822,0,0.101458,0,0.014109,-1.000000,0.591770,0,0,0.712518,0.806812,0
L,0.000000,0.038645,0.231534,0.203963,0.000000,0,-1.000000,0,0.000000,0.177793,0.118686,0,0,-0.999726,0.444837,0
R,0.481134,0.506107,-0.124263,0.000000,-1.000000,0,-1.000000,0,-0.039203,0.482807,-1.000000,0,0,0.887081,1.000000,0


### Algorithm Implementation

In [95]:
Qtable_sarsa
# S . . .
# . H . H
# . . . H 
# H . . E


,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
D,0.422522,-0.864915,0.399269,-0.343900,0.469387,0,0.736161,0,-1.000000,0.742783,0.877696,0,0,0.000000,0.000000,0
U,0.000000,0.000000,0.000000,0.000000,0.320790,0,0.025330,0,0.397491,-1.000000,0.508752,0,0,0.597623,0.791927,0
L,0.000000,0.286282,-0.056014,-0.018457,0.000000,0,-0.612580,0,0.000000,0.515090,0.533387,0,0,-0.999994,0.715918,0
R,0.054473,0.098052,-0.049321,0.000000,-0.999987,0,-0.468559,0,0.502781,0.608707,-0.999989,0,0,0.890780,1.000000,0


In [ ]:

for i in range(no_of_episodes_sarsa):    
    # reset environment to start state
    env1.reset()  

    # Initiialise main step state
    main_step_state = env1.current_state()

    while env1.is_terminal(env1.current_state()) == False:
        main_step_state = env1.current_state()


        while env1.is_terminal(env1.current_state()) == False:
            # Choose action
            sub_step_state = env1.current_state()
            state_formatted = str(sub_step_state[0]) + str(sub_step_state[1])
            sub_step_action = epsilon_soft(Qtable_sarsa, env1, epsilon_sarsa, sub_step_state)
            
            # Move to check next state choices
            env1.move(sub_step_action)

            # Retrieve reward for taking specific action
            reward = env1.get_rewards()
            # Retrieve new state and next action choice
            new_sub_step_state = env1.current_state()
            new_state_formatted = str(new_sub_step_state[0]) + str(new_sub_step_state[1])

            print("Sub State: {}\nSub Action: {}\n\n".format(sub_step_state, sub_step_action))
            if new_sub_step_state in env1.actions:
                new_sub_step_action = epsilon_soft(Qtable_sarsa, env1, epsilon_sarsa, new_sub_step_state)
                
                # Update Q table with percentage of rewards for taking specific action at specific state to arrive at new state
                    # q_value = Qtable_sarsa.at[sub_step_action, state_formatted]
                    # q_prime_value = Qtable_sarsa.at[new_sub_step_action, new_state_formatted] 
                    # q_value += (alpha_sarsa * (reward + (gamma_sarsa * q_prime_value) - q_value))

                Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha_sarsa * (reward + (gamma_sarsa * Qtable_sarsa.at[new_sub_step_action, new_state_formatted]) - Qtable_sarsa.at[sub_step_action, state_formatted]))

            else: 
                Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha_sarsa * (reward + (gamma_sarsa * Qtable_sarsa.at[new_sub_step_action, new_state_formatted]) - Qtable_sarsa.at[sub_step_action, state_formatted]))

                continue

        env1.set_state(main_step_state)
        main_step_action = epsilon_soft(Qtable_sarsa, env1, epsilon_sarsa, main_step_state)
        env1.move(main_step_action)    
        print("Main state: {}\nMain Action: {}".format(main_step_state, main_step_action))   

        

Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: U


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: R


Sub State: (0, 1)
Sub Action: L


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: U


Sub State: (1, 0)
Sub Action: R


Main state: (0, 0)
Main Action: D
Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: R


Sub State: (2, 1)
Sub Action: D


Sub State: (3, 1)
Sub Action: R


Sub State: (3, 2)
Sub Action: L


Sub State: (3, 1)
Sub Action: U


Sub State: (2, 1)
Sub Action: L


Sub State: (2, 0)
Sub Action: D


Main state: (1, 0)
Main Action: D
Sub State: (2, 0)
Sub Action: U


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: R


Main state: (2, 0)
Main Action: U
Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: U


Sub State: (1,

In [ ]:

for i in range(no_of_episodes_sarsa):    
    # reset environment to start state
    env.reset()  

    # Initiialise main step state
    main_step_state = env.current_state()

    while env.is_terminal(env.current_state()) == False:
        main_step_state = env.current_state()


        while env.is_terminal(env.current_state()) == False:
            # Choose action
            sub_step_state = env.current_state()
            state_formatted = str(sub_step_state[0]) + str(sub_step_state[1])
            sub_step_action = epsilon_soft(Qtable_sarsa, env, epsilon_sarsa, sub_step_state)
            
            # Move to check next state choices
            env.move(sub_step_action)

            # Retrieve reward for taking specific action
            reward = env.get_rewards()
            # Retrieve new state and next action choice
            new_sub_step_state = env.current_state()
            new_state_formatted = str(new_sub_step_state[0]) + str(new_sub_step_state[1])

            print("Sub State: {}\nSub Action: {}\n\n".format(sub_step_state, sub_step_action))
            if new_sub_step_state in env.actions:
                new_sub_step_action = epsilon_soft(Qtable_sarsa, env, epsilon_sarsa, new_sub_step_state)
                
                # Update Q table with percentage of rewards for taking specific action at specific state to arrive at new state
                    # q_value = Qtable_sarsa.at[sub_step_action, state_formatted]
                    # q_prime_value = Qtable_sarsa.at[new_sub_step_action, new_state_formatted] 
                    # q_value += (alpha_sarsa * (reward + (gamma_sarsa * q_prime_value) - q_value))

                Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha_sarsa * (reward + (gamma_sarsa * Qtable_sarsa.at[new_sub_step_action, new_state_formatted]) - Qtable_sarsa.at[sub_step_action, state_formatted]))

            else: 
                Qtable_sarsa.loc[sub_step_action, state_formatted] += (alpha_sarsa * (reward + (gamma_sarsa * Qtable_sarsa.at[new_sub_step_action, new_state_formatted]) - Qtable_sarsa.at[sub_step_action, state_formatted]))

                continue

        env.set_state(main_step_state)
        main_step_action = epsilon_soft(Qtable_sarsa, env, epsilon_sarsa, main_step_state)
        env.move(main_step_action)    
        print("Main state: {}\nMain Action: {}".format(main_step_state, main_step_action))   

        

NameError: name 'env' is not defined

_______________________________________________________________________________________________________________________________

# Qlearning with an ϵ-greedy behavior policy

In [ ]:
Qtable_qlearning = create_qtable(env.rows, env.cols, action_space)
Qtable_qlearning

,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
D,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
U,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
L,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
R,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Defining parameters**

In [ ]:
epsilon_qlearning = 0.2 # soft policy epsilon probability
gamma_qlearning = 0.9 # discount rate
alpha_qlearning = 0.2
no_of_episodes_qlearning = 100# total number of episodes

In [ ]:
# Qlearning Algorithm

# Create new Qtable for SARSA
Qtable_qlearning = Qtable_qlearning = create_qtable(env.rows, env.cols, action_space)

for i in range(no_of_episodes_qlearning):    
    # reset environment to start state
    env.reset()  


    # Choose action at state
        # Look forward one step and update Q table
        # Choose action based on new Qtable
        # Look forward one step based on new state
        # Choose new action based on new state
        # Update Q table
        ## End when hit terminal state
    
    # Rechoose action based on new Q table
        # Repeat

    # Initiialise main step state
    main_step_state = env.current_state()

    while env.is_terminal(env.current_state()) == False:
        main_step_state = env.current_state()


        while env.is_terminal(env.current_state()) == False:
            # Choose action
            sub_step_state = env.current_state()
            state_formatted = str(sub_step_state[0]) + str(sub_step_state[1])
            sub_step_action = epsilon_soft(Qtable_qlearning, env, epsilon_qlearning, sub_step_state)
            
            # Move to check next state choices
            env.move(sub_step_action)

            # Retrieve reward for taking specific action
            reward = env.get_rewards()
            # Retrieve new state and next action choice
            new_sub_step_state = env.current_state()
            new_state_formatted = str(new_sub_step_state[0]) + str(new_sub_step_state[1])

            print("Sub State: {}\nSub Action: {}\n\n".format(sub_step_state, sub_step_action))
            if new_sub_step_state in env.actions:
                new_sub_step_action = epsilon_soft(Qtable_qlearning, env, epsilon_qlearning, new_sub_step_state)
                
                # Update Q table with percentage of rewards for taking specific action at specific state to arrive at new state
                    # q_value = Qtable_sarsa.at[sub_step_action, state_formatted]
                    # q_prime_value = Qtable_sarsa.at[new_sub_step_action, new_state_formatted] 
                    # q_value += (alpha_sarsa * (reward + (gamma_sarsa * q_prime_value) - q_value))

                Qtable_qlearning.loc[sub_step_action, state_formatted] += (alpha_qlearning * (reward + (gamma_qlearning * Qtable_qlearning[new_state_formatted].max()) - Qtable_qlearning.at[sub_step_action, state_formatted]))

            else: 
                Qtable_qlearning.loc[sub_step_action, state_formatted] += (alpha_qlearning * (reward + (gamma_qlearning * Qtable_qlearning[new_state_formatted].max()) - Qtable_qlearning.at[sub_step_action, state_formatted]))

                continue

        env.set_state(main_step_state)
        main_step_action = epsilon_soft(Qtable_qlearning, env, epsilon_qlearning, main_step_state)
        env.move(main_step_action)    
        print("Main state: {}\nMain Action: {}".format(main_step_state, main_step_action))   

        

Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: R


Sub State: (0, 1)
Sub Action: L


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: R


Main state: (0, 0)
Main Action: R
Sub State: (0, 1)
Sub Action: L


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: R


Sub State: (0, 1)
Sub Action: D


Main state: (0, 1)
Main Action: D
Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: U


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: U


Sub State: (0, 0)
Sub Action: D


Sub State: (1, 0)
Sub Action: D


Sub State: (2, 0)
Sub Action: R


Sub State: (2, 1)
Sub Action: U


Main state: (0, 0)
Main Action: R
Sub State: (0, 1)
Sub Action: D


Main state: (0, 1)
Main Action: R
Sub State: (0, 2)
Sub Action: L


Sub State: (0,

In [ ]:
Qtable_qlearning
# S . . .
# . H . H
# . . . H 
# H . . E


,00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
D,0.433645,-0.992621,0.729000,-0.790285,0.239818,0,0.810000,0,-0.360000,0.809982,0.900000,0,0,0.000000,0.000000,0
U,0.000000,0.000000,0.000000,0.000000,0.522756,0,0.646548,0,0.004041,-0.360000,0.727513,0,0,0.617821,0.809428,0
L,0.000000,0.521068,0.563698,0.655317,0.000000,0,-0.964816,0,0.000000,0.205407,0.726900,0,0,-0.737856,0.809818,0
R,0.590490,0.656100,0.577192,0.000000,-0.360000,0,-0.994097,0,0.592851,0.000000,-0.994097,0,0,0.900000,1.000000,0
